# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load csv into dataframe

weather_Data = pd.read_csv("Weather_Output.csv")

weather_Data.head()


,Unnamed: 0,City,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,17.80,60,0,5.69,AU,1644710528
1,1,Castro,-24.7911,-50.0119,17.73,69,32,2.87,BR,1644710260
2,2,Vanimo,-2.6741,141.3028,27.46,80,70,1.27,PG,1644710528
3,3,Makkah al Mukarramah,21.4267,39.8261,21.80,30,0,5.98,SA,1644710029
4,4,Constitución,-35.3333,-72.4167,16.01,82,1,3.13,CL,1644710456


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
# gmaps.configure(api_key=g_key)
# Store Lat and Lng into  locations 
locations = weather_Data[["Latitude", "Longitude"]].astype(float)
# use humidity as weight
humidity = weather_Data["Humidity"].astype(float)

In [4]:
# Create a Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius = 3, opacity=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# set ideal weather conditions between 70 and 80 degrees
weather_DF = weather_Data.loc[(weather_Data["Temp"] >= 15) & (weather_Data["Temp"] <= 25) & (weather_Data["Wind Speed"] <= 10) & (weather_Data["Cloudiness"]>= 20) & (weather_Data["Cloudiness"]<= 30)].dropna()                                          
# drop any rows that don't contain all three conditions
weather_DF = weather_DF.dropna()
#Display dataframe
weather_DF

,Unnamed: 0,City,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,Huarmey,-10.0681,-78.1522,22.57,76,26,3.44,PE,1644710531
51,51,Arraial do Cabo,-22.9661,-42.0278,24.01,84,22,2.72,BR,1644710542
58,58,Ulladulla,-35.3500,150.4667,22.47,51,20,2.05,AU,1644710544
67,67,Saint-Philippe,-21.3585,55.7679,23.66,83,23,4.48,RE,1644710546
89,89,Palmares,-8.6833,-35.5917,23.98,89,24,2.05,BR,1644710553
136,136,George Town,5.4112,100.3354,24.80,84,20,2.57,MY,1644710302
232,232,Ponta do Sol,32.6667,-17.1000,16.68,71,27,2.06,PT,1644710593
236,236,Taltal,-25.4000,-70.4833,20.13,80,21,2.68,CL,1644710594
282,282,Pacifica,37.6138,-122.4869,22.83,50,20,2.06,US,1644710607
353,353,Batemans Bay,-35.7167,150.1833,21.42,51,20,2.47,AU,1644710627


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# store into variable named hotel_df
hotel_df = weather_DF[["City","Latitude","Longitude", "Country"]]
# set up a parameters dictionary
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}
# set base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_df["Hotel Name"] = ""
# loop through all hotels in within the paramaters
for index,row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City"]
    
    # add location parameter
    params["location"] = f"{lat},{lng}"
    
    print(f"Processing results for {city}...")
    response = requests.get(base_url, params=params).json()
          
    # gather hotel search results
    results = response['results']
          
# Store the first Hotel result into the DataFrame.
    try: 
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except:
        print(f"Could not find result for {city}.")
        pass

<ipython-input-6-af0c41329641>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


Processing results for Huarmey...
Could not find result for Huarmey.
Processing results for Arraial do Cabo...
Could not find result for Arraial do Cabo.
Processing results for Ulladulla...
Could not find result for Ulladulla.
Processing results for Saint-Philippe...
Could not find result for Saint-Philippe.
Processing results for Palmares...
Could not find result for Palmares.
Processing results for George Town...
Could not find result for George Town.
Processing results for Ponta do Sol...
Could not find result for Ponta do Sol.
Processing results for Taltal...
Could not find result for Taltal.
Processing results for Pacifica...
Could not find result for Pacifica.
Processing results for Batemans Bay...
Could not find result for Batemans Bay.
Processing results for Sibut...
Could not find result for Sibut.
Processing results for Santa María del Oro...
Could not find result for Santa María del Oro.
Processing results for Wau...
Could not find result for Wau.


In [7]:
hotel_df

,City,Latitude,Longitude,Country,Hotel Name
13,Huarmey,-10.0681,-78.1522,PE,
51,Arraial do Cabo,-22.9661,-42.0278,BR,
58,Ulladulla,-35.3500,150.4667,AU,
67,Saint-Philippe,-21.3585,55.7679,RE,
89,Palmares,-8.6833,-35.5917,BR,
136,George Town,5.4112,100.3354,MY,
232,Ponta do Sol,32.6667,-17.1000,PT,
236,Taltal,-25.4000,-70.4833,CL,
282,Pacifica,37.6138,-122.4869,US,
353,Batemans Bay,-35.7167,150.1833,AU,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Display figure
fig.add_layer(hotel_markers)
fig

Figure(layout=FigureLayout(height='420px'))